In [69]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pyupbit

In [70]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda, Input, GlobalAveragePooling1D, Bidirectional
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [71]:
def get_model(input_shape):
    input = Input(shape=input_shape)
    x = LSTM(128, return_sequences=True, activation='tanh', dropout=0.2)(input)
    x = LSTM(64, return_sequences=True, activation='tanh', dropout=0.2)(x)
    x = LSTM(32, return_sequences=True, activation='tanh', dropout=0.2)(x)
    x = GlobalAveragePooling1D()(x)
    output = Dense(1)(x)
    model = Model(input, output)
    return model

WINDOW_SIZE = 6
feature_n = 6

Input_shape = (WINDOW_SIZE, feature_n)
model = get_model(Input_shape)
model.load_weights('../checkpoints/ckeckpointer.ckpt')

model.summary()

ValueError: Shapes (64, 1) and (32, 1) are incompatible

In [82]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pyupbit
from sklearn.preprocessing import MinMaxScaler

import os
from glob import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split



class Data_preprocess():
    def __init__(self, args):
        if not hasattr(args, 'data') or args.data is None:
            print(args)
            self.data, self.label, self.dataset = self.preprocess(
                pyupbit.get_ohlcv(ticker=args.ticker, interval=args.interval, to=args.to, count=args.count))
        else :
            self.data, self.label, self.dataset = self.csv_parsing(args.data)

    def MinMax(self, dataset_df):
        norm = MinMaxScaler()
        norm_dataset = norm.fit_transform(dataset_df)
        return pd.DataFrame(norm_dataset, columns=list(dataset_df.columns))

    def add_after10(self, dataset_df):
        after10 = np.zeros_like(self.norm_dataset['close'])
        for i in range(len(dataset_df['close']) - 1):
            after10[i] = dataset_df['close'][i + 1]
        return after10

    def drop_feature(self, dataset_df):
        # index(시간) 제거
        dataset_df = dataset_df.reset_index(drop=True)
        # value 제거
        dataset_df = dataset_df.drop(columns=['value'])
        return dataset_df

    def add_avgPrice(self, dataset_df):
        return (dataset_df['high'] + dataset_df['low'] +
                dataset_df['open'] + dataset_df['close']) // 4

    def preprocess(self, dataset, latest=False):

        # drop feature
        #dataset_df = self.drop_feature(dataset)
        dataset_df = dataset

        # avg_price 추가
        dataset_df['avg_price'] = self.add_avgPrice(dataset_df)

        if latest == True:
            # 가장 예전 데이터 삭제 - norm이랑 original 둘 다 적용
            self.dataset = self.dataset.drop([self.dataset.index[0]]).drop(columns=['after10'])
            self.norm_dataset = self.norm_dataset.drop([self.norm_dataset.index[0]])

            # ori dataset에 추가
            self.dataset = pd.concat([self.dataset, dataset_df])
            self.dataset = self.dataset.reset_index(drop=True)

            # min max 정규화 (MinMaxScaler) 적용
            self.norm_dataset = self.MinMax(self.dataset)

            # after10 추가
            self.dataset['after10'] = self.add_after10(self.dataset)


        else:
            # min max 정규화 (MinMaxScaler) 적용
            self.norm_dataset = self.MinMax(dataset_df)

            # after10 추가
            dataset_df['after10'] = self.add_after10(dataset_df)

        # 예측될 값(label)인 10분 후 가격
        self.norm_dataset['after10'] = self.add_after10(self.norm_dataset)

        return self.norm_dataset.drop(columns=['after10']), self.norm_dataset['after10'], dataset_df

    def csv_parsing(self, data_path):
        merge_df = pd.DataFrame()
        data_folders = glob(os.path.join(data_path, '*'))

        for data_folder in tqdm(data_folders):
            data_csvs = glob(os.path.join(data_folder,'*.csv'))

            for data_csv in data_csvs :
                csv_df = pd.read_csv(data_csv).drop(columns=["Unnamed: 0"])
                merge_df = pd.concat([merge_df, csv_df], ignore_index=True)

        return self.preprocess(merge_df)


    # dataset에 window 적용
    def windowed_dataset(self, data, label, window_size, batch_size):
        sliced_data = tf.data.Dataset.from_tensor_slices(data)
        sliced_data = sliced_data.window(window_size, shift=1, stride=1, drop_remainder=True)
        sliced_data = sliced_data.flat_map(lambda x: x.batch(window_size))

        sliced_label = tf.data.Dataset.from_tensor_slices(label[window_size:])

        sliced_dataset = tf.data.Dataset.zip((sliced_data, sliced_label))

        return sliced_dataset.batch(batch_size).prefetch(1)





In [83]:
def get_model_BiLSTM(input_shape):
    input = Input(shape=input_shape)
    x = Bidirectional(LSTM(128, return_sequences=True, activation='tanh', dropout=0.3))(input)
    x = Bidirectional(LSTM(64, return_sequences=True, activation='tanh', dropout=0.3))(x)
    x = Bidirectional(LSTM(32, return_sequences=True, activation='tanh', dropout=0.3))(x)
    x = GlobalAveragePooling1D()(x)
    output = Dense(1)(x)
    model = Model(input, output)
    return model

def get_model(input_shape):
    input = Input(shape=input_shape)
    x = LSTM(128, return_sequences=True, activation='tanh', dropout=0.2)(input)
    x = LSTM(64, return_sequences=True, activation='tanh', dropout=0.2)(x)
    x = LSTM(32, return_sequences=True, activation='tanh', dropout=0.2)(x)
    x = GlobalAveragePooling1D()(x)
    output = Dense(1)(x)
    model = Model(input, output)
    return model


WINDOW_SIZE = 6
feature_n = 6

Bi_WINDOW_SIZE = 12
Bi_feature_n = 7

Input_shape = (Bi_WINDOW_SIZE, Bi_feature_n)
model = get_model_BiLSTM(Input_shape)
model.load_weights('./checkpoints/BiLSTM/ckeckpointer.ckpt')

model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 12, 7)]           0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 12, 256)           139264    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 12, 128)           164352    
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 12, 64)            41216     
_________________________________________________________________
global_average_pooling1d_5 ( (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 344,897
Trainable params: 344,897
Non-trainable params: 0
_____________________________________________________

In [96]:
from easydict import EasyDict
import datetime

options = {
    'ticker' : 'KRW-BTC',
    'interval' : 'minute10',
    'to' : datetime.datetime.now().strftime('%Y-%m-%d %H:%M'),
    'count' : 2000
}

args = EasyDict(options)

processed_data = Data_preprocess(args)

{'ticker': 'KRW-BTC', 'interval': 'minute10', 'to': '2021-11-16 09:48', 'count': 2000}


In [97]:
print(processed_data.data)

          open      high       low     close    volume     value  avg_price
0     0.067044  0.097650  0.075403  0.094142  0.186938  0.172054   0.082207
1     0.094051  0.087999  0.071589  0.091501  0.160973  0.147976   0.085060
2     0.095279  0.083604  0.079218  0.086973  0.146444  0.134660   0.084987
3     0.086874  0.102236  0.092812  0.086690  0.293749  0.270809   0.090837
4     0.092540  0.088859  0.057311  0.051222  0.153978  0.141464   0.071040
...        ...       ...       ...       ...       ...       ...        ...
1995  0.519358  0.512899  0.500636  0.478068  0.402771  0.392546   0.506514
1996  0.478376  0.512803  0.485966  0.495236  0.438786  0.426934   0.496773
1997  0.495562  0.492070  0.471394  0.448920  0.283932  0.275858   0.480457
1998  0.448631  0.467132  0.460929  0.467975  0.306766  0.297327   0.464432
1999  0.468366  0.464934  0.473252  0.461655  0.131033  0.127077   0.470354

[2000 rows x 7 columns]


In [98]:
print(processed_data.label)

0       0.091501
1       0.086973
2       0.086690
3       0.051222
4       0.038487
          ...   
1995    0.495236
1996    0.448920
1997    0.467975
1998    0.461655
1999    0.000000
Name: after10, Length: 2000, dtype: float64


In [99]:
def inference(lstm_model, processed_data, window_size, batch_size):
    dataset = processed_data.windowed_dataset(processed_data.data, processed_data.label, window_size, batch_size)
    pred = lstm_model.predict(dataset)
    pred = pred[:, 0]
    
    return pred

In [100]:
Bi_WINDOW_SIZE = 12

WINDOW_SIZE = 6
BATCH_SIZE=1

pred = inference(model, processed_data, Bi_WINDOW_SIZE, BATCH_SIZE)


In [101]:
actual = processed_data.label[WINDOW_SIZE:].reset_index(drop=True)

In [102]:
actual

0       0.017357
1       0.005660
2       0.020470
3       0.009716
4       0.000000
          ...   
1989    0.495236
1990    0.448920
1991    0.467975
1992    0.461655
1993    0.000000
Name: after10, Length: 1994, dtype: float64

In [103]:
pred_rate_val = []
act_rate_val = []
accuracy = []

for i in range(0, len(pred) - 2) :
    rate_pred = pred[i + 1]/pred[i]
    pred_rate_val.append(rate_pred)
    
    if rate_pred < 1:
        # Down
        pred_tmp = 0
    else:
        # Up
        pred_tmp = 1
        
    rate_act = actual[i + 2] / actual[i + 1]
    act_rate_val.append(rate_act)
    
    if rate_act < 1:
        # Down
        act_tmp = 0
    else:
        # Up
        act_tmp = 1
        
    if pred_tmp == act_tmp :
        accuracy.append(1)
    else :
        accuracy.append(0)

c:\users\quhb2\anaconda3\envs\coin\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()


In [104]:
# 정확도
sum(accuracy) / len(accuracy)

0.56797583081571

In [65]:
import numpy as np

np_pred_rate_val = np.array(pred_rate_val)
np_accuracy = np.array(accuracy)

In [66]:
import copy

print(np_pred_rate_val)
print(np_accuracy.shape)

a_np_pred_rate_val =  copy.deepcopy(np_pred_rate_val)
b_np_pred_rate_val =  copy.deepcopy(np_pred_rate_val)

[1.0266085  1.0222495  1.0386864  1.0072212  1.0012504  1.002732
 1.0104018  1.0259616  1.0147438  1.010896   1.0004454  1.0074378
 0.9969732  0.9970859  0.99593    1.0015895  1.0062711  0.9989115
 0.9928641  0.9923726  0.99101746 0.9839887  0.9988457  1.0043885
 1.0098144  1.0138528  1.0195811  1.0270455  1.0192032  1.0209723
 1.0058188  0.9971594  0.998505   0.992818   0.9875472  0.98794496
 0.98872304 0.9849087  0.9958638  0.999887   0.9993528  0.9973917
 0.9947604  1.0014883  1.0095834  1.002961   1.0003883  0.9994338
 0.99592376 0.99026334 0.9884415  0.98716575 0.99206346 0.99592197
 1.0043924  1.004332   1.0054196  1.002476   0.9959359  1.0008674
 0.9976586  0.997627   1.0072576  1.0162153  1.0111749  1.002346
 0.9974126  0.99525076 0.9896278  0.98797965 0.98788077 0.9803982
 0.97631073 0.9712009  0.96584904 0.96451575 0.98727345 0.9935657
 0.99218225 0.9967635  1.0579534  1.0326477  1.0121143  1.0094633
 1.010294   1.0092967  1.0065305  1.0024788  1.002419   0.9940647
 0.9806071

In [67]:
a_np_pred_rate_val[ np_accuracy[:]==1 ] = 0
a_np_pred_rate_val

array([0.        , 1.0222495 , 0.        , 1.0072212 , 1.0012504 ,
       0.        , 1.0104018 , 1.0259616 , 0.        , 0.        ,
       1.0004454 , 0.        , 0.9969732 , 0.        , 0.99593   ,
       0.        , 1.0062711 , 0.        , 0.9928641 , 0.9923726 ,
       0.99101746, 0.9839887 , 0.        , 1.0043885 , 0.        ,
       1.0138528 , 0.        , 1.0270455 , 1.0192032 , 1.0209723 ,
       1.0058188 , 0.9971594 , 0.998505  , 0.        , 0.        ,
       0.98794496, 0.        , 0.9849087 , 0.        , 0.999887  ,
       0.9993528 , 0.9973917 , 0.        , 0.        , 1.0095834 ,
       0.        , 1.0003883 , 0.9994338 , 0.        , 0.99026334,
       0.        , 0.98716575, 0.        , 0.        , 1.0043924 ,
       0.        , 0.        , 1.002476  , 0.        , 0.        ,
       0.        , 0.997627  , 1.0072576 , 1.0162153 , 1.0111749 ,
       1.002346  , 0.        , 0.        , 0.        , 0.98797965,
       0.        , 0.        , 0.97631073, 0.9712009 , 0.96584

In [68]:
b_np_pred_rate_val[ np_accuracy[:]==False ] = 0
b_np_pred_rate_val

array([1.0266085 , 0.        , 1.0386864 , 0.        , 0.        ,
       1.002732  , 0.        , 0.        , 1.0147438 , 1.010896  ,
       0.        , 1.0074378 , 0.        , 0.9970859 , 0.        ,
       1.0015895 , 0.        , 0.9989115 , 0.        , 0.        ,
       0.        , 0.        , 0.9988457 , 0.        , 1.0098144 ,
       0.        , 1.0195811 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.992818  , 0.9875472 ,
       0.        , 0.98872304, 0.        , 0.9958638 , 0.        ,
       0.        , 0.        , 0.9947604 , 1.0014883 , 0.        ,
       1.002961  , 0.        , 0.        , 0.99592376, 0.        ,
       0.9884415 , 0.        , 0.99206346, 0.99592197, 0.        ,
       1.004332  , 1.0054196 , 0.        , 0.9959359 , 1.0008674 ,
       0.9976586 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.9974126 , 0.99525076, 0.9896278 , 0.        ,
       0.98788077, 0.9803982 , 0.        , 0.        , 0.     

In [45]:
np_pred_rate_val[ np_accuracy[:]==1 ] = 0

In [20]:
print(992 - np.count_nonzero(np_pred_rate_val))

495


In [21]:
c = 0
for i in np_pred_rate_val:
    if i == 0:
        c += 1
c

495

In [22]:
c/993

0.4984894259818731